In [1]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import numpy as np
import ray
from ray.rllib import agents
from tqdm.notebook import tqdm
import random
from ray.rllib.policy.policy import Policy
from gym.spaces import Discrete, Box
from ray.rllib.agents.ppo import PPOTrainer
from functools import partial
from ray.tune.logger import pretty_print
from ray.rllib.agents.ppo.ppo_tf_policy import PPOTFPolicy
from ray.rllib.models import ModelCatalog


import ray
from ray import tune
from ray.tune import track

import math
import gym

from gym_compete_to_rllib import created_envs, env_name, create_env, env_name_rllib

import os
os.environ['DISPLAY'] = ':0'

In [2]:
def ray_init():
    ray.shutdown()
    return ray.init(ignore_reinit_error=True,
                    temp_dir='/scratch/sergei/tmp') # Skip or set to ignore if already called

ray_init()


2020-07-21 04:54:20,040	INFO resource_spec.py:223 -- Starting Ray with 106.2 GiB memory available for workers and up to 49.53 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-07-21 04:54:20,829	INFO services.py:1193 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '128.32.175.10',
 'raylet_ip_address': '128.32.175.10',
 'redis_address': '128.32.175.10:15454',
 'object_store_address': '/scratch/sergei/tmp/session_2020-07-21_04-54-20_036388_3839/sockets/plasma_store',
 'raylet_socket_name': '/scratch/sergei/tmp/session_2020-07-21_04-54-20_036388_3839/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/scratch/sergei/tmp/session_2020-07-21_04-54-20_036388_3839'}

2020-07-21 04:54:22,351	WARNING worker.py:1115 -- The dashboard on node svm failed with the following error:
Traceback (most recent call last):
  File "/scratch/sergei/better-adversarial-defenses/ray/python/ray/dashboard/dashboard.py", line 960, in <module>
    metrics_export_address=metrics_export_address)
  File "/scratch/sergei/better-adversarial-defenses/ray/python/ray/dashboard/dashboard.py", line 513, in __init__
    build_dir = setup_static_dir(self.app)
  File "/scratch/sergei/better-adversarial-defenses/ray/python/ray/dashboard/dashboard.py", line 414, in setup_static_dir
    "&& npm run build)", build_dir)
FileNotFoundError: [Errno 2] Dashboard build directory not found. If installing from source, please follow the additional steps required to build the dashboard(cd python/ray/dashboard/client && npm ci && npm run build): '/scratch/sergei/better-adversarial-defenses/ray/python/ray/dashboard/client/build'



In [3]:
tf.keras.backend.set_floatx('float32')

In [4]:
env_cls = create_env
env_config = {'with_video': True}#True}

def build_trainer_config(restore_state=None, train_policies=None, config=None):
    """Build configuration for 1 run."""
    obs_space = env_cls(env_config).observation_space
    act_space = env_cls(env_config).action_space

    policy_template = "player_%d"

    def get_agent_config(agent_id):
        agent_config_pretrained = (PPOTFPolicy, obs_space, act_space, {
            'model': {
                        "custom_model": "GymCompetePretrainedModel",
                        "custom_model_config": {
                            "agent_id": agent_id - 1,
                            "env_name": env_name,
                            "model_config": {},
                            "name": "model_%s" % (agent_id - 1)
                        },           
                        
                    },
            
            "framework": "tfe",
        })
        
        agent_config_from_scratch = (PPOTFPolicy, obs_space, act_space, {
                    "model": {
                        "use_lstm": False,
                        "fcnet_hiddens": [64, 64],
                        #"custom_action_dist": "DiagGaussian",
                    },
                    "framework": "tfe",
                })
        
        if agent_id == 1:
            return agent_config_from_scratch
        elif agent_id == 2:
            return agent_config_pretrained
        else:
            raise KeyError("Wrong agent id %s" % agent_id)

    N_POLICIES = 2

    policies = {policy_template % i: get_agent_config(i) for i in range(1, 1  + N_POLICIES)}
    policies_keys = list(sorted(policies.keys()))

    def select_policy(agent_id):
        assert agent_id in ["player_1", "player_2"]
        agent_ids = ["player_1", "player_2"]
        
        # selecting the corresponding policy (only for 2 policies)
        return policies_keys[agent_ids.index(agent_id)]

        # randomly choosing an opponent
        # return np.random.choice(list(policies.keys()))
    
    if train_policies is None:
        train_policies = list(policies.keys())
        
    for k in train_policies:
        assert k in policies.keys()

    config = {
        "env": env_name_rllib,
        "env_config": env_config,
    #    "gamma": 0.9,
      "num_workers": 0,
    #  "num_envs_per_worker": 10,
    #   "rollout_fragment_length": 10,
       "train_batch_size": config['train_batch_size'],
        "multiagent": {
            "policies_to_train": train_policies,
            "policies": policies,
            "policy_mapping_fn": select_policy,
        },
        "framework": "tfe",
        #"train_batch_size": 512
        #"num_cpus_per_worker": 2
    }
    return config


def build_trainer(restore_state=None, train_policies=None, config=None):
    """Create a RPS trainer for 2 agents, restore state, and only train specific policies."""
    
    print("Using config")
    print(config)
    cls = PPOTrainer
    trainer = cls(config=config)
    env = trainer.workers.local_worker().env
    if restore_state is not None:
        trainer.restore_from_object(restore_state)
    return trainer

stats = []

def train(trainer, stop_iters, do_track=True):
    """Train the agents and return the state of the trainer."""
    for _ in range(stop_iters):
        results = trainer.train()
        print(pretty_print(results))
        if do_track:
            track.log(**results)
        global stats
        stats.append(results)
    o = trainer.save_to_object()
    return o

    


# try changing learning rate
config = {'train_batch_size': 256}

config['train_steps'] = 10

# ['humanoid_blocker', 'humanoid'],
config['train_policies'] = []#'player_1']
#config['num_workers'] = 32


print(config)
restore_state=None
rl_config = build_trainer_config(restore_state=restore_state,
                          train_policies=config['train_policies'],
                          config=config)
trainer = build_trainer(restore_state=None, config=rl_config)


{'train_batch_size': 256, 'train_steps': 10, 'train_policies': []}


GLFW error: 65544, desc: b'X11: RandR gamma ramp support seems broken'


Creating agent humanoid_blocker
Reading agent XML from: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Creating agent humanoid
Reading agent XML from: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Scene XML path: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/world_body.humanoid_body.humanoid_body.xml
Created Scene with agents
Creating agent humanoid_blocker
Reading agent XML from: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Creating agent humanoid
Reading agent XML from: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Scene XML path: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/world_body.humanoid_body.humanoid_body.xml
Create

2020-07-21 04:54:43,204	INFO trainer.py:598 -- Executing eagerly, with eager_tracing=False
2020-07-21 04:54:43,205	INFO trainer.py:628 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


Using config
{'env': 'YouShallNotPassHumans-v0_rllib', 'env_config': {'with_video': True}, 'num_workers': 0, 'train_batch_size': 256, 'multiagent': {'policies_to_train': [], 'policies': {'player_1': (<class 'ray.rllib.policy.tf_policy_template.PPOTFPolicy'>, Box(380,), Box(17,), {'model': {'use_lstm': False, 'fcnet_hiddens': [64, 64]}, 'framework': 'tfe'}), 'player_2': (<class 'ray.rllib.policy.tf_policy_template.PPOTFPolicy'>, Box(380,), Box(17,), {'model': {'custom_model': 'GymCompetePretrainedModel', 'custom_model_config': {'agent_id': 1, 'env_name': 'multicomp/YouShallNotPassHumans-v0', 'model_config': {}, 'name': 'model_1'}}, 'framework': 'tfe'})}, 'policy_mapping_fn': <function build_trainer_config.<locals>.select_policy at 0x7f7717042170>}, 'framework': 'tfe'}
Creating agent humanoid_blocker
Reading agent XML from: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Creating agent humanoid
Reading agent XML from: /scra

2020-07-21 04:54:44,655	WARNING util.py:37 -- Install gputil for GPU system monitoring.


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 380)]        0                                            
__________________________________________________________________________________________________
observation_preprocessing_layer (None, 380)          761         input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 64)           24384       observation_preprocessing_layer[0
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 64)           4160        dense[0][0]                      
____________________________________________________________________________________________

In [5]:
env_cls()

TypeError: 'NoneType' object is not subscriptable

In [8]:
tune.loguniform

<function ray.tune.sample.loguniform(min_bound, max_bound, base=10)>

In [6]:
trainer.config

{'num_workers': 0,
 'num_envs_per_worker': 1,
 'rollout_fragment_length': 200,
 'sample_batch_size': -1,
 'batch_mode': 'truncate_episodes',
 'num_gpus': 0,
 'train_batch_size': 256,
 'model': {'conv_filters': None,
  'conv_activation': 'relu',
  'fcnet_activation': 'tanh',
  'fcnet_hiddens': [256, 256],
  'free_log_std': False,
  'no_final_linear': False,
  'vf_share_layers': True,
  'use_lstm': False,
  'max_seq_len': 20,
  'lstm_cell_size': 256,
  'lstm_use_prev_action_reward': False,
  'state_shape': None,
  'framestack': True,
  'dim': 84,
  'grayscale': False,
  'zero_mean': True,
  'custom_model': None,
  'custom_model_config': {},
  'custom_action_dist': None,
  'custom_preprocessor': None,
  'custom_options': -1},
 'optimizer': {},
 'gamma': 0.99,
 'horizon': None,
 'soft_horizon': False,
 'no_done_at_end': False,
 'env_config': {'with_video': True},
 'env': 'YouShallNotPassHumans-v0_rllib',
 'normalize_actions': False,
 'clip_rewards': None,
 'clip_actions': True,
 'preproces

In [15]:
env = create_env(config={'with_video': False})

Creating agent humanoid_blocker
Reading agent XML from: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Creating agent humanoid
Reading agent XML from: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Scene XML path: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/world_body.humanoid_body.humanoid_body.xml
Created Scene with agents


In [16]:
env.reset()

{'player_1': array([-9.32710249e-01,  6.95427847e-02,  1.45666368e+00,  9.97072535e-01,
        -5.13707862e-02,  5.61041578e-02,  7.72820599e-03, -9.36057776e-02,
         5.49654006e-02, -9.29106058e-02,  1.13921151e-02,  2.91018816e-02,
        -2.83979130e-02,  2.63614763e-02, -2.62624640e-02,  9.60320431e-03,
        -9.83927686e-03,  7.26802975e-02,  1.12754140e-02, -4.86952067e-02,
        -8.62042804e-02, -3.93374184e-02, -9.38066434e-02, -4.81061004e-02,
        -2.15188954e-01,  1.44322431e-01,  5.57476580e-02, -2.90068343e-03,
        -1.05695891e-01, -5.17121895e-02, -3.71732076e-03, -7.33339077e-02,
        -1.07224233e-01,  7.51855207e-02, -5.59667248e-02, -9.54777046e-02,
        -5.26137892e-02,  5.30313616e-02, -7.81256268e-02, -1.66563527e-01,
         2.04433247e-01, -3.86015833e-02,  5.47227976e-02, -6.57769617e-02,
        -2.04718834e-02, -3.61235890e-02, -6.84215077e-02,  2.20450794e+00,
         2.17186677e+00,  1.00412146e-01, -3.13784764e-02, -2.20787314e-01,


In [19]:
env.step({'player_1': np.zeros(380), 'player_2': np.zeros(380)})

({'player_1': array([-9.39808841e-01,  7.39562418e-02,  1.45406302e+00,  9.97036832e-01,
         -5.16322090e-02,  5.66815568e-02,  6.23479142e-03, -8.58902598e-02,
          4.45365572e-02, -9.31284658e-02,  1.14381325e-02,  2.69046436e-02,
         -3.59658580e-02, -7.26028531e-04, -2.24370903e-02,  9.82796781e-03,
         -3.10664467e-02,  2.70188504e-02,  8.68489840e-03, -4.22359319e-02,
         -8.99328920e-02, -3.64478519e-02, -9.69855953e-02, -4.99961009e-02,
         -2.52029448e-01,  1.49253393e-01, -2.32011649e-01, -2.59225816e-02,
          8.41853158e-02, -1.14977499e-01,  3.18957916e-01, -5.70065192e-01,
          3.31320373e-02, -1.32683991e-02, -5.43304664e-02, -4.27107097e-02,
         -1.00685282e+00,  1.49245766e-01, -3.09021263e-02, -5.53327677e-01,
         -1.79934933e+00, -8.45789919e-02,  2.61221658e-01, -1.49226937e-01,
          1.28913172e-01, -1.32254648e-01, -7.72207923e-02,  2.20824900e+00,
          2.17155733e+00,  9.88490817e-02, -2.97004152e-02, -2.0

In [5]:
import pickle, json, codecs
ck = json.load(open('/home/sergei/ray_results/./adversarial/train_one_3_2020-07-17_21-34-39m9qon2mn/checkpoint_0/checkpoint', 'r'))
w = codecs.decode(ck['weights'].encode(), 'base64')
trainer.set_weights(pickle.loads(w))

FileNotFoundError: [Errno 2] No such file or directory: '/home/sergei/ray_results/./adversarial/train_one_3_2020-07-17_21-34-39m9qon2mn/checkpoint_0/checkpoint'

In [6]:
trainer.config

{'num_workers': 0,
 'num_envs_per_worker': 1,
 'rollout_fragment_length': 200,
 'sample_batch_size': -1,
 'batch_mode': 'truncate_episodes',
 'num_gpus': 0,
 'train_batch_size': 256,
 'model': {'conv_filters': None,
  'conv_activation': 'relu',
  'fcnet_activation': 'tanh',
  'fcnet_hiddens': [256, 256],
  'free_log_std': False,
  'no_final_linear': False,
  'vf_share_layers': True,
  'use_lstm': False,
  'max_seq_len': 20,
  'lstm_cell_size': 256,
  'lstm_use_prev_action_reward': False,
  'state_shape': None,
  'framestack': True,
  'dim': 84,
  'grayscale': False,
  'zero_mean': True,
  'custom_model': None,
  'custom_model_config': {},
  'custom_action_dist': None,
  'custom_preprocessor': None,
  'custom_options': -1},
 'optimizer': {},
 'gamma': 0.99,
 'horizon': None,
 'soft_horizon': False,
 'no_done_at_end': False,
 'env_config': {'with_video': True},
 'env': 'YouShallNotPassHumans-v0_rllib',
 'normalize_actions': False,
 'clip_rewards': None,
 'clip_actions': True,
 'preproces

In [32]:
train(trainer, config['train_steps'], do_track=False)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.
custom_metrics: {}
date: 2020-07-17_21-43-20
done: false
episode_len_mean: 152.0
episode_reward_max: -0.3383652470135523
episode_reward_mean: -0.48690533688773474
episode_reward_min: -0.6354454267619172
episodes_this_iter: 2
episodes_total: 2
experiment_id: c6ee96fcb50744afb96244184244c6bf
hostname: svm
info:
  learner:
    player_1:
      cur_kl_coeff: 0.20000000298023224
      cur_lr: 4.999999873689376e-05
      en

FileNotFoundError: [Errno 2] No such file or directory: '/home/sergei/ray_results/PPO_YouShallNotPassHumans-v0_rllib_2020-07-17_21-41-32btsd_9s9/tmpoccvyew6save_to_object'

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.hist(np.array([y for x in stats for y in x['hist_stats']['policy_player_1_reward']]).flatten())

In [33]:
[x.close() for x in created_envs]

[None, None, None, None, None, None, None, None, None]

In [ ]:
trainer.evaluate()

In [ ]:
return

In [ ]:
from load_gym_compete_policy import get_policy_value_nets

In [ ]:
obs = env_cls(env_config).reset()

In [ ]:
acts = [trainer.compute_action(obs['player_1'], policy_id='player_1') for _ in range(1000)]

In [ ]:
np.mean(acts, axis=0)

In [ ]:
nets = get_policy_value_nets(env_name, 0)
policy_net_orig = nets['policy_mean_logstd_flat']

In [ ]:
m = trainer.get_policy('player_1').model
m.policy_net(obs['player_1'].reshape(1, -1))

In [ ]:
nets['policy_mean_logstd'](obs['player_1'].reshape(1, -1))

In [ ]:
mean, logstd = tf.split(policy_net_orig(obs['player_1'].reshape(1, -1)), 2, axis=1)

In [ ]:
np.mean([np.clip(np.random.normal(loc=mean[0], scale=np.exp(logstd[0]), size=(17,)), -0.4, 0.4) for _ in range(1000)], axis=0)

In [ ]:
np.mean([trainer.compute_action(obs['player_1'], policy_id='player_1') for _ in range(1000)], axis=0)

In [ ]:
trainer.compute_action(obs['player_1'], policy_id='player_1')

In [ ]:
env_cls(env_config).action_space.high

In [ ]:
trainer.config

In [16]:
import codecs

codecs.encode(pickle.dumps(np.array([1])), 'base64').decode()

'gANjbnVtcHkuY29yZS5tdWx0aWFycmF5Cl9yZWNvbnN0cnVjdApxAGNudW1weQpuZGFycmF5CnEB\nSwCFcQJDAWJxA4dxBFJxBShLAUsBhXEGY251bXB5CmR0eXBlCnEHWAIAAABpOHEIiYiHcQlScQoo\nSwNYAQAAADxxC05OTkr/////Sv////9LAHRxDGKJQwgBAAAAAAAAAHENdHEOYi4=\n'